In [18]:
import numpy as np
import pandas as pd
import re
import math

beer = pd.read_csv("~/data400_share/beer.csv")

beer = beer.dropna(subset=['review/appearance'])
beer = beer.dropna(subset=['review/aroma'])
beer = beer.dropna(subset=['review/overall'])
beer = beer.dropna(subset=['review/palate'])
beer = beer.dropna(subset=['review/taste'])
beer = beer.dropna(subset=['review/palate'])
beer = beer.dropna(subset=['review/text'])
beer = beer.iloc[:, list(range(0,13))]

beer.head(1)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ..."


In [19]:
#s = "12oz bottle into 8oz snifter\t\tDeep ruby"
#s = s.replace('\t', '')

beer["review/text"] = beer["review/text"].str.replace('[^\w\s]',' ')
beer["review/text"] = beer["review/text"].str.replace('\t', ' ')
beer.head(1)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ..."


In [20]:
beer["stopwordcount"] = 0
beer['charcount'] = 0
beer['cursewordcount'] = 0
beer['uniquecursewordcount'] = 0
cursewords = ["anal",
"anus",
"arse",
"ass",
"ballsack",
"balls",
"bastard",
"bitch",
"biatch",
"bloody",
"blowjob",
"bollock",
"bollok",
"boner",
"boob",
"bugger",
"bum",
"butt",
"clitoris",
"cock",
"coon",
"crap",
"cunt",
"damn",
"dick",
"dildo",
"dyke",
"fag",
"feck",
"fellate",
"fellatio",
"felching",
"fuck",
"fudgepacker",
"flange",
"goddamn",
"damn",
"hell",
"homo",
"jerk",
"jizz",
"knobend",
"labia",
"lmao",
"lmfao",
"muff",
"nigger",
"nigga",
"omg",
"penis",
"piss",
"poop",
"prick",
"pube",
"pussy",
"queer",
"scrotum",
"sex",
"shit",
"sh1t",
"slut",
"smegma",
"spunk",
"tit",
"tosser",
"turd",
"twat",
"vagina",
"wank",
"whore",
"wtf"]
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [21]:
beer["review/text"] = beer["review/text"].str.split(" ")

In [22]:
beer.head(1)
#beer = beer.drop(beer.index[3171])
beer.index = pd.RangeIndex(len(beer.index))
beer.index = range(len(beer.index))

In [23]:
for i in list(range(beer.shape[0])):
    currentlist = beer.at[i, "review/text"]
    currentlist = list(map(str.lower,currentlist))
    beer.at[i, "charcount"] = (sum(len(i) for i in currentlist))
    results1 = {}
    results2 = {}
    for j in stopwords:
        results1[j] = currentlist.count(j)
    for k in cursewords:
        results2[k] = currentlist.count(k)
    beer.at[i, "stopwordcount"] = sum(results1.values())
    beer.at[i, "cursewordcount"] = sum(results2.values())
    beer.at[i, "uniquecursewordcount"] = sum(1 for i in results2.values() if i >= 1)

In [25]:
beer.head(10)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,stopwordcount,charcount,cursewordcount,uniquecursewordcount
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,"[Pours, a, clouded, gold, with, a, thin, white...","{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ...",42,415,0,0
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,"[12oz, bottle, into, 8oz, snifter, , , Deep, r...","{'min': 38, 'hour': 23, 'mday': 8, 'sec': 58, ...",42,450,0,0
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,"[First, enjoyed, at, the, brewpub, about, 2, y...","{'min': 7, 'hour': 18, 'mday': 26, 'sec': 2, '...",21,312,0,0
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,"[First, thing, I, noticed, after, pouring, fro...","{'min': 7, 'hour': 1, 'mday': 20, 'sec': 5, 'y...",27,312,0,0
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,"[A, , pours, an, amber, with, a, one, finger, ...","{'min': 51, 'hour': 6, 'mday': 12, 'sec': 48, ...",49,383,0,0
5,14095,10.0,21436,14,YouEnjoyMyStout,Russian Imperial Stout,4.0,4.0,4.0,4.0,4.5,"[served, in, a, snifter, , on, tap, at, CBC, ,...","{'min': 16, 'hour': 1, 'mday': 20, 'sec': 9, '...",67,717,0,0
6,13759,12.6,10001,14,Benevolence,American Wild Ale,4.0,4.5,4.0,4.0,4.0,"[my, 100th, review, on, tap, , mini, snifter, ...","{'min': 1, 'hour': 18, 'mday': 8, 'sec': 12, '...",47,564,0,0
7,3914,8.4,52211,14879,Frog's Hollow Double Pumpkin Ale,Pumpkin Ale,4.0,4.0,4.0,4.0,4.0,"[One, of, the, better, pumpkin, ales, I, ve, t...","{'min': 32, 'hour': 20, 'mday': 9, 'sec': 40, ...",92,1139,0,0
8,47953,9.0,34361,394,Stoudt's Barrel Aged Old Abominable,English Barleywine,4.5,4.0,4.5,4.5,4.5,"[A, snifter, filled, , several, times, over, ,...","{'min': 26, 'hour': 3, 'mday': 16, 'sec': 13, ...",104,1252,1,1
9,7047,6.2,32676,1386,One Hop Wonder IPA,American IPA,2.0,2.0,2.0,2.5,2.0,"[Brewed, with, five, hop, additions, , this, a...","{'min': 26, 'hour': 5, 'mday': 16, 'sec': 41, ...",46,554,0,0
